# Regressão Logística

## Objetivos

- Compreender os fundamentos da regressão logística
- Implementar regressão logística do zero
- Usar scikit-learn para classificação
- Interpretar resultados e probabilidades

## Pré-requisitos

- Fundamentos de ML
- Regressão Linear
- Conceitos básicos de probabilidade


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

np.random.seed(42)
plt.style.use("default")

## 1. Conceitos Fundamentais

A regressão logística é um algoritmo de classificação que usa a função logística (sigmoide) para modelar a probabilidade de uma classe.

### Função Sigmoide

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

Onde $z = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n$


In [ ]:
# Visualizar a função sigmoide
z = np.linspace(-10, 10, 100)
sigmoid = 1 / (1 + np.exp(-z))

plt.figure(figsize=(10, 6))
plt.plot(z, sigmoid, "b-", linewidth=2, label="Função Sigmoide")
plt.axhline(y=0.5, color="r", linestyle="--", alpha=0.7, label="Threshold = 0.5")
plt.xlabel("z (entrada linear)")
plt.ylabel("σ(z) (probabilidade)")
plt.title("Função Sigmoide")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("Propriedades da função sigmoide:")
print(f"σ(-∞) ≈ {sigmoid[0]:.3f}")
print(f"σ(0) = {sigmoid[50]:.3f}")
print(f"σ(+∞) ≈ {sigmoid[-1]:.3f}")

## 2. Dados de Exemplo

Vamos gerar um dataset sintético para classificação binária.


In [ ]:
# Gerar dados sintéticos
X, y = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    random_state=42,
)

print(f"Formato dos dados: X={X.shape}, y={y.shape}")
print(f"Classes: {np.unique(y)}")
print(f"Distribuição das classes: {np.bincount(y)}")

In [ ]:
# Visualizar os dados
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap="RdYlBu", alpha=0.7)
plt.colorbar(scatter)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Dataset de Classificação Binária")
plt.grid(True, alpha=0.3)
plt.show()

## 3. Implementação do Zero

Vamos implementar a regressão logística sem usar bibliotecas.


In [ ]:
class LogisticRegressionCustom:
    def __init__(self, learning_rate=0.01, max_iterations=1000):
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.weights = None
        self.bias = None
        self.costs = []

    def _add_bias(self, X):
        """Adiciona coluna de bias (intercepto)"""
        return np.c_[np.ones(X.shape[0]), X]

    def _sigmoid(self, z):
        """Função sigmoide com estabilidade numérica"""
        z = np.clip(z, -250, 250)  # Evitar overflow
        return 1 / (1 + np.exp(-z))

    def _cost_function(self, h, y):
        """Função de custo (log-likelihood negativa)"""
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    def fit(self, X, y):
        """Treinar o modelo"""
        # Inicializar pesos
        n_features = X.shape[1]
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Gradiente descendente
        for i in range(self.max_iterations):
            # Forward pass
            z = X.dot(self.weights) + self.bias
            h = self._sigmoid(z)

            # Calcular custo
            cost = self._cost_function(h, y)
            self.costs.append(cost)

            # Calcular gradientes
            dw = X.T.dot(h - y) / y.size
            db = (h - y).mean()

            # Atualizar pesos
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict_proba(self, X):
        """Predizer probabilidades"""
        z = X.dot(self.weights) + self.bias
        return self._sigmoid(z)

    def predict(self, X):
        """Predizer classes"""
        return (self.predict_proba(X) >= 0.5).astype(int)


# Teste da implementação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Treinar modelo customizado
model_custom = LogisticRegressionCustom(learning_rate=0.1, max_iterations=1000)
model_custom.fit(X_train, y_train)

# Fazer previsões
y_pred_custom = model_custom.predict(X_test)
accuracy_custom = accuracy_score(y_test, y_pred_custom)

print(f"Acurácia do modelo customizado: {accuracy_custom:.3f}")

In [ ]:
# Visualizar a convergência
plt.figure(figsize=(10, 6))
plt.plot(model_custom.costs, "b-", linewidth=2)
plt.xlabel("Iterações")
plt.ylabel("Custo (Log-Loss)")
plt.title("Convergência do Gradiente Descendente")
plt.grid(True, alpha=0.3)
plt.show()

print(f"Custo inicial: {model_custom.costs[0]:.3f}")
print(f"Custo final: {model_custom.costs[-1]:.3f}")

## 4. Usando Scikit-Learn

Agora vamos comparar com a implementação do scikit-learn.


In [ ]:
# Treinar modelo do scikit-learn
model_sklearn = LogisticRegression(random_state=42)
model_sklearn.fit(X_train, y_train)

# Fazer previsões
y_pred_sklearn = model_sklearn.predict(X_test)
y_proba_sklearn = model_sklearn.predict_proba(X_test)

accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print(f"Acurácia do scikit-learn: {accuracy_sklearn:.3f}")
print(f"Acurácia do modelo customizado: {accuracy_custom:.3f}")

# Comparar pesos
print(f"\nPesos (customizado): {model_custom.weights}")
print(f"Pesos (sklearn): {model_sklearn.coef_[0]}")
print(f"Bias (customizado): {model_custom.bias:.3f}")
print(f"Bias (sklearn): {model_sklearn.intercept_[0]:.3f}")

## 5. Visualização da Fronteira de Decisão


In [ ]:
def plot_decision_boundary(X, y, model, title="Fronteira de Decisão"):
    """Plotar fronteira de decisão"""
    h = 0.01
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    mesh_points = np.c_[xx.ravel(), yy.ravel()]

    if hasattr(model, "predict_proba"):
        if hasattr(model, "classes_"):  # sklearn
            Z = model.predict_proba(mesh_points)[:, 1]
        else:  # modelo customizado
            Z = model.predict_proba(mesh_points)
    else:
        Z = model.predict(mesh_points)

    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(12, 5))

    # Subplot 1: Fronteira de decisão
    plt.subplot(1, 2, 1)
    plt.contourf(xx, yy, Z, levels=50, alpha=0.8, cmap="RdYlBu")
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap="RdYlBu", edgecolors="black")
    plt.colorbar(scatter)
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title(f"{title} - Probabilidades")

    # Subplot 2: Linha de decisão
    plt.subplot(1, 2, 2)
    plt.contour(xx, yy, Z, levels=[0.5], colors="black", linestyles="--", linewidths=2)
    scatter = plt.scatter(X[:, 0], X[:, 1], c=y, cmap="RdYlBu", edgecolors="black")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title(f"{title} - Linha de Decisão")

    plt.tight_layout()
    plt.show()


# Visualizar ambos os modelos
plot_decision_boundary(X_test, y_test, model_sklearn, "Scikit-Learn")
plot_decision_boundary(X_test, y_test, model_custom, "Implementação Customizada")

## 6. Métricas de Avaliação


In [ ]:
# Matriz de confusão
def plot_confusion_matrix(y_true, y_pred, title="Matriz de Confusão"):
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=["Classe 0", "Classe 1"],
        yticklabels=["Classe 0", "Classe 1"],
    )
    plt.xlabel("Predito")
    plt.ylabel("Real")
    plt.title(title)
    plt.show()

    return cm


cm = plot_confusion_matrix(y_test, y_pred_sklearn)

# Relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_sklearn))

In [ ]:
# Análise das probabilidades
probabilities = model_sklearn.predict_proba(X_test)[:, 1]

plt.figure(figsize=(12, 5))

# Subplot 1: Distribuição das probabilidades
plt.subplot(1, 2, 1)
plt.hist(probabilities[y_test == 0], bins=30, alpha=0.7, label="Classe 0", color="blue")
plt.hist(probabilities[y_test == 1], bins=30, alpha=0.7, label="Classe 1", color="red")
plt.axvline(x=0.5, color="black", linestyle="--", label="Threshold")
plt.xlabel("Probabilidade Predita")
plt.ylabel("Frequência")
plt.title("Distribuição das Probabilidades")
plt.legend()

# Subplot 2: Curva ROC (simplificada)
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, probabilities)
roc_auc = auc(fpr, tpr)

plt.subplot(1, 2, 2)
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--", label="Random")
plt.xlabel("Taxa de Falsos Positivos")
plt.ylabel("Taxa de Verdadeiros Positivos")
plt.title("Curva ROC")
plt.legend()

plt.tight_layout()
plt.show()

print(f"AUC-ROC: {roc_auc:.3f}")

## 7. Regressão Logística Multiclasse

Exemplo com mais de 2 classes usando One-vs-Rest.


In [ ]:
# Gerar dados multiclasse
X_multi, y_multi = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42,
)

X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42
)

# Treinar modelo multiclasse
model_multi = LogisticRegression(random_state=42, multi_class="ovr")
model_multi.fit(X_train_multi, y_train_multi)

y_pred_multi = model_multi.predict(X_test_multi)
accuracy_multi = accuracy_score(y_test_multi, y_pred_multi)

print(f"Acurácia multiclasse: {accuracy_multi:.3f}")
print(f"Classes: {model_multi.classes_}")

In [ ]:
# Visualizar classificação multiclasse
plot_decision_boundary(
    X_test_multi, y_test_multi, model_multi, "Classificação Multiclasse"
)

# Matriz de confusão multiclasse
cm_multi = plot_confusion_matrix(
    y_test_multi, y_pred_multi, "Matriz de Confusão - Multiclasse"
)

## 8. Resumo e Conclusões

### Pontos-chave da Regressão Logística:

1. **Função Sigmoide**: Mapeia qualquer valor real para [0,1]
2. **Interpretabilidade**: Coeficientes representam log-odds
3. **Probabilidades**: Fornece probabilidades calibradas
4. **Flexibilidade**: Funciona para classificação binária e multiclasse
5. **Eficiência**: Rápido para treinar e predizer

### Vantagens:

- Simples e interpretável
- Não requer tuning de hiperparâmetros
- Menos propenso a overfitting
- Fornece probabilidades

### Limitações:

- Assume relação linear entre features e log-odds
- Sensível a outliers
- Requer features com escalas similares
- Não captura interações complexas

### Próximos Passos:

- Support Vector Machines
- Árvores de Decisão
- Ensemble Methods
